# [ 3-3. 운영비용 설정 ]

## 1. 필요한 모듈 import 및 기본 설정
### 1-1. 필요한 모듈 import

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

from m01_assumption import assumption
from m02_index import index
from m04_operating_income import operating_income

### 1-2. assumption 데이터 확인

In [2]:
assumption['room_operating_cost']

{'청소세탁비': {'TypeA': 10000, 'TypeB': 12000, 'TypeC': 20000},
 '수도광열비': {'TypeA': 5000, 'TypeB': 6000, 'TypeC': 10000, 'Overhead': 3000000},
 '예약수수료율': {'TypeA': 0.03, 'TypeB': 0.03, 'TypeC': 0.03}}

In [3]:
assumption['management_cost']

{'광고홍보비예산': {'amount': 200000000},
 '광고홍보비': {1: 0.15,
  2: 0.15,
  3: 0.05,
  4: 0.05,
  5: 0.05,
  6: 0.05,
  7: 0.1,
  8: 0.1,
  9: 0.05,
  10: 0.05,
  11: 0.05,
  12: 0.15},
 '기타운영비': {'amount': 10000000}}

In [4]:
assumption['salary_cost']

{'employee_count': {'객실운영팀_정규직': 5,
  '객실운영팀_임시직': 2,
  '경영지원팀_임원': 1,
  '경영지원팀_정규직': 3,
  '마케팅팀_정규직': 2,
  '시설관리팀_정규직': 2,
  '시설관리팀_임시직': 1},
 'annual_salary': {'객실운영팀_정규직': 36000000,
  '객실운영팀_임시직': 30000000,
  '경영지원팀_임원': 80000000,
  '경영지원팀_정규직': 36000000,
  '마케팅팀_정규직': 36000000,
  '시설관리팀_정규직': 36000000,
  '시설관리팀_임시직': 30000000}}

### 1-3. 빈 operating_cost 딕셔너리 설정

In [5]:
operating_cost = {}

## 2. 객실운영비 설정

In [6]:
operating_cost['객실운영비'] = {}

### 2-1. 청소세탁비

In [7]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['TypeA'] = int(
            operating_income['TypeA'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['청소세탁비']['TypeA'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeB'] = int(
            operating_income['TypeB'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['청소세탁비']['TypeB'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeC'] = int(
            operating_income['TypeC'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['청소세탁비']['TypeC'] *
            index['연간인상률'].loc[dt, '운영비']
        )
    else:
        dct['TypeA'] = 0
        dct['TypeB'] = 0
        dct['TypeC'] = 0

    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

operating_cost['객실운영비']['청소세탁비'] = pd.DataFrame(data, index=index['model'])

### 2-2. 수도광열비

In [8]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['TypeA'] = int(
            operating_income['TypeA'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['수도광열비']['TypeA'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeB'] = int(
            operating_income['TypeB'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['수도광열비']['TypeB'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeC'] = int(
            operating_income['TypeC'].loc[dt, '판매객실수'] * 
            assumption['room_operating_cost']['수도광열비']['TypeC'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['Overhead'] = int(
            assumption['room_operating_cost']['수도광열비']['Overhead'] *
            index['연간인상률'].loc[dt, '운영비']
        )
    else:
        dct['TypeA'] = 0
        dct['TypeB'] = 0
        dct['TypeC'] = 0
        dct['Overhead'] = 0

    dct['Total'] = (
        dct['TypeA'] + 
        dct['TypeB'] + 
        dct['TypeC'] +
        dct['Overhead']
    )
    data.append(dct)

operating_cost['객실운영비']['수도광열비'] = pd.DataFrame(data, index=index['model'])

### 2-3. 예약수수료

In [9]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['TypeA'] = int(
            operating_income['TypeA'].loc[dt, '객실수입'] * 
            assumption['room_operating_cost']['예약수수료율']['TypeA']
        )
        dct['TypeB'] = int(
            operating_income['TypeB'].loc[dt, '객실수입'] * 
            assumption['room_operating_cost']['예약수수료율']['TypeB']
        )
        dct['TypeC'] = int(
            operating_income['TypeC'].loc[dt, '객실수입'] * 
            assumption['room_operating_cost']['예약수수료율']['TypeC']
        )
    else:
        dct['TypeA'] = 0
        dct['TypeB'] = 0
        dct['TypeC'] = 0
        
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

operating_cost['객실운영비']['예약수수료'] = pd.DataFrame(data, index=index['model'])

### 2-4. SubTotal
* '청소세탁비', '수도광열비', '예약수수료'의 사용처 구분에 따라 분류된 수치를 그에 따른 하위 항목('TypeA', 'TypeB', 등)을 기준으로 재분류해주는 함수 작성.

* 즉, '객실운영비 - 청소세탁비 - TypeA', '객실운영비 - 청소세탁비 - TypeB' 등의 구조로 비용이 분류되어 있는 상태에서 '객실운영비 - TypeA', '객실운영비 - TypeB'의 구분값과 같이 하위 항목 기준으로 재분류하여 계산함.

In [10]:
def subtotal(df, param):
    subtotal = 0
    for key in df.keys():
        try:
            subtotal = subtotal + df[key][param]
        except KeyError:
            pass
    return subtotal


In [11]:
operating_cost['객실운영비']['TypeA'] = subtotal(operating_cost['객실운영비'], 'TypeA')
operating_cost['객실운영비']['TypeB'] = subtotal(operating_cost['객실운영비'], 'TypeB')
operating_cost['객실운영비']['TypeC'] = subtotal(operating_cost['객실운영비'], 'TypeC')
operating_cost['객실운영비']['Overhead'] = subtotal(operating_cost['객실운영비'], 'Overhead')
operating_cost['객실운영비']['Total'] = subtotal(operating_cost['객실운영비'], 'Total')

## 3. 관리운영비

In [12]:
operating_cost['관리운영비'] = {}

### 3-1. 관리운영비

In [13]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['광고홍보비'] = int(
            assumption['management_cost']['광고홍보비예산']['amount'] *
            assumption['management_cost']['광고홍보비'][dt.month] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['기타운영비'] = int(
            assumption['management_cost']['기타운영비']['amount'] *
            index['연간인상률'].loc[dt, '운영비']
        )
    else:
        dct['광고홍보비'] = 0
        dct['기타운영비'] = 0
        
    dct['Total'] = dct['광고홍보비'] + dct['기타운영비']
    data.append(dct)

operating_cost['관리운영비']['관리운영비'] = pd.DataFrame(data, index=index['model'])


### 3-2. SubTotal

In [14]:
operating_cost['관리운영비']['Total'] = subtotal(operating_cost['관리운영비'], 'Total')

## 4. 인건비

In [15]:
operating_cost['인건비'] = {}

### 4-1. 객실운영팀

In [16]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['정규직'] = int(
            assumption['salary_cost']['employee_count']['객실운영팀_정규직'] *
            assumption['salary_cost']['annual_salary']['객실운영팀_정규직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
        dct['임시직'] = int(
            assumption['salary_cost']['employee_count']['객실운영팀_임시직'] *
            assumption['salary_cost']['annual_salary']['객실운영팀_임시직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
    else:
        dct['정규직'] = 0
        dct['임시직'] = 0
        
    dct['Total'] = dct['정규직'] + dct['임시직']
    data.append(dct)
operating_cost['인건비']['객실운영팀'] = pd.DataFrame(data, index=index['model'])

### 4-2. 경영지원팀

In [17]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['정규직'] = int(
            assumption['salary_cost']['employee_count']['경영지원팀_정규직'] *
            assumption['salary_cost']['annual_salary']['경영지원팀_정규직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
        dct['임원'] = int(
            assumption['salary_cost']['employee_count']['경영지원팀_임원'] *
            assumption['salary_cost']['annual_salary']['경영지원팀_임원'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
    else:
        dct['정규직'] = 0
        dct['임원'] = 0
        
    dct['Total'] = dct['정규직'] + dct['임원']
    data.append(dct)
operating_cost['인건비']['경영지원팀'] = pd.DataFrame(data, index=index['model'])

### 4-3. 마케팅팀

In [18]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['정규직'] = int(
            assumption['salary_cost']['employee_count']['마케팅팀_정규직'] *
            assumption['salary_cost']['annual_salary']['마케팅팀_정규직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
    else:
        dct['정규직'] = 0
        
    dct['Total'] = dct['정규직']
    data.append(dct)
operating_cost['인건비']['마케팅팀'] = pd.DataFrame(data, index=index['model'])

### 4-4. 시설관리팀

In [19]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['정규직'] = int(
            assumption['salary_cost']['employee_count']['시설관리팀_정규직'] *
            assumption['salary_cost']['annual_salary']['시설관리팀_정규직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
        dct['임시직'] = int(
            assumption['salary_cost']['employee_count']['시설관리팀_임시직'] *
            assumption['salary_cost']['annual_salary']['시설관리팀_임시직'] *
            index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
            index['연간인상률'].loc[dt, '인건비']
        )
    else:
        dct['정규직'] = 0
        dct['임시직'] = 0
        
    dct['Total'] = dct['정규직'] + dct['임시직']
    data.append(dct)
operating_cost['인건비']['시설관리팀'] = pd.DataFrame(data, index=index['model'])

### 4-5. SubTotal

In [20]:
operating_cost['인건비']['정규직'] = subtotal(operating_cost['인건비'], '정규직')
operating_cost['인건비']['임시직'] = subtotal(operating_cost['인건비'], '임시직')
operating_cost['인건비']['임원'] = subtotal(operating_cost['인건비'], '임원')
operating_cost['인건비']['Total'] = subtotal(operating_cost['인건비'], 'Total')

## 5. operating_cost.py 파일 작성
* subtotal 함수는 general_function 모듈로 이동

* 나머지 코드들은 operating_cost.py 파일로 이동

* general_function 모듈에서 subtotal 함수 import하는 코드 추가

In [21]:
# m05_operating_cost.py

import pandas as pd

from m00_general_function import subtotal
from m01_assumption import assumption
from m02_index import index
from m04_operating_income import operating_income


operating_cost = {}

#### 1. 객실운영비
operating_cost['객실운영비'] = {}

## 1-1. 청소세탁비
data = []
for dt in index['model']:
    dct = {}
    dct['TypeA'] = int(
        operating_income['TypeA'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['청소세탁비']['TypeA'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeB'] = int(
        operating_income['TypeB'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['청소세탁비']['TypeB'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeC'] = int(
        operating_income['TypeC'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['청소세탁비']['TypeC'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

operating_cost['객실운영비']['청소세탁비'] = pd.DataFrame(data, index=index['model'])


## 1-2. 수도광열비
data = []
for dt in index['model']:
    dct = {}
    dct['TypeA'] = int(
        operating_income['TypeA'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['수도광열비']['TypeA'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeB'] = int(
        operating_income['TypeB'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['수도광열비']['TypeB'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeC'] = int(
        operating_income['TypeC'].loc[dt, '판매객실수'] * 
        assumption['room_operating_cost']['수도광열비']['TypeC'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['Overhead'] = int(
        assumption['room_operating_cost']['수도광열비']['Overhead'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['Total'] = (
        dct['TypeA'] + 
        dct['TypeB'] + 
        dct['TypeC'] +
        dct['Overhead']
    )
    data.append(dct)

operating_cost['객실운영비']['수도광열비'] = pd.DataFrame(data, index=index['model'])


## 1-3. 예약수수료
data = []
for dt in index['model']:
    dct = {}
    dct['TypeA'] = int(
        operating_income['TypeA'].loc[dt, '객실수입'] * 
        assumption['room_operating_cost']['예약수수료율']['TypeA']
    )
    dct['TypeB'] = int(
        operating_income['TypeB'].loc[dt, '객실수입'] * 
        assumption['room_operating_cost']['예약수수료율']['TypeB']
    )
    dct['TypeC'] = int(
        operating_income['TypeC'].loc[dt, '객실수입'] * 
        assumption['room_operating_cost']['예약수수료율']['TypeC']
    )
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

operating_cost['객실운영비']['예약수수료'] = pd.DataFrame(data, index=index['model'])


## 1-4. SubTotal
operating_cost['객실운영비']['TypeA'] = subtotal(operating_cost['객실운영비'], 'TypeA')
operating_cost['객실운영비']['TypeB'] = subtotal(operating_cost['객실운영비'], 'TypeB')
operating_cost['객실운영비']['TypeC'] = subtotal(operating_cost['객실운영비'], 'TypeC')
operating_cost['객실운영비']['Overhead'] = subtotal(operating_cost['객실운영비'], 'Overhead')
operating_cost['객실운영비']['Total'] = subtotal(operating_cost['객실운영비'], 'Total')


#### 2. 관리운영비
operating_cost['관리운영비'] = {}

## 2-1. 관리운영비
data = []
for dt in index['model']:
    dct = {}
    dct['광고홍보비'] = int(
        assumption['management_cost']['광고홍보비예산']['amount'] *
        assumption['management_cost']['광고홍보비'][dt.month] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['기타운영비'] = int(
        assumption['management_cost']['기타운영비']['amount'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['Total'] = dct['광고홍보비'] + dct['기타운영비']
    data.append(dct)

operating_cost['관리운영비']['관리운영비'] = pd.DataFrame(data, index=index['model'])

## 2-2. SubTotal
operating_cost['관리운영비']['Total'] = subtotal(operating_cost['관리운영비'], 'Total')


#### 3. 인건비
operating_cost['인건비'] = {}

## 3-1. 객실운영팀
data = []
for dt in index['model']:
    dct = {}
    dct['정규직'] = int(
        assumption['salary_cost']['employee_count']['객실운영팀_정규직'] *
        assumption['salary_cost']['annual_salary']['객실운영팀_정규직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['임시직'] = int(
        assumption['salary_cost']['employee_count']['객실운영팀_임시직'] *
        assumption['salary_cost']['annual_salary']['객실운영팀_임시직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['Total'] = dct['정규직'] + dct['임시직']
    data.append(dct)
operating_cost['인건비']['객실운영팀'] = pd.DataFrame(data, index=index['model'])

## 3-2. 경영지원팀
data = []
for dt in index['model']:
    dct = {}
    dct['정규직'] = int(
        assumption['salary_cost']['employee_count']['경영지원팀_정규직'] *
        assumption['salary_cost']['annual_salary']['경영지원팀_정규직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['임원'] = int(
        assumption['salary_cost']['employee_count']['경영지원팀_임원'] *
        assumption['salary_cost']['annual_salary']['경영지원팀_임원'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['Total'] = dct['정규직'] + dct['임원']
    data.append(dct)
operating_cost['인건비']['경영지원팀'] = pd.DataFrame(data, index=index['model'])

## 3-3. 마케팅팀
data = []
for dt in index['model']:
    dct = {}
    dct['정규직'] = int(
        assumption['salary_cost']['employee_count']['마케팅팀_정규직'] *
        assumption['salary_cost']['annual_salary']['마케팅팀_정규직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['Total'] = dct['정규직']
    data.append(dct)
operating_cost['인건비']['마케팅팀'] = pd.DataFrame(data, index=index['model'])

## 3-4. 시설관리팀
data = []
for dt in index['model']:
    dct = {}
    dct['정규직'] = int(
        assumption['salary_cost']['employee_count']['시설관리팀_정규직'] *
        assumption['salary_cost']['annual_salary']['시설관리팀_정규직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['임시직'] = int(
        assumption['salary_cost']['employee_count']['시설관리팀_임시직'] *
        assumption['salary_cost']['annual_salary']['시설관리팀_임시직'] *
        index['days'].loc[dt, '월간일수'] / index['days'].loc[dt, '연간일수'] *
        index['연간인상률'].loc[dt, '인건비']
    )
    dct['Total'] = dct['임시직']
    data.append(dct)
operating_cost['인건비']['시설관리팀'] = pd.DataFrame(data, index=index['model'])

## 3-5. SubTotal
operating_cost['인건비']['정규직'] = subtotal(operating_cost['인건비'], '정규직')
operating_cost['인건비']['임시직'] = subtotal(operating_cost['인건비'], '임시직')
operating_cost['인건비']['임원'] = subtotal(operating_cost['인건비'], '임원')
operating_cost['인건비']['Total'] = subtotal(operating_cost['인건비'], 'Total')